In [1]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("article_new.csv")
print(len(df))
df = df.iloc[:, 3:5]
df.head()

83


,Insight,Insight Type
0,The Utah Jazz have helped Lauri Markkanen unlo...,"Single Player Performance, Strategy and Tactic"
1,Division and conference makes it easy for Mave...,Strategy and Tactic
2,Kevin Durant had to leave the game after Jimmy...,Accident
3,"The Lakers hustled more, but the Mavericks sho...",Whole Game Review
4,"Without Kevin Durant, Kyrie Irving is aggressi...","Single Player Performance, Strategy and Tactic"


In [4]:
types = df['Insight Type'].values
types = [insight_type.split(", ") for insight_type in types]
type_dictionary = {}
for type_list in types:
    for type_ in type_list:
        # print(type_)
        if type_ not in type_dictionary:
            type_dictionary[type_] = 1
        else:
            type_dictionary[type_] += 1

type_dictionary

{'Single Player Performance': 51,
 'Strategy and Tactic': 26,
 'Accident': 6,
 'Whole Game Review': 22,
 'About Referee': 4}

In [5]:
from sklearn.preprocessing import MultiLabelBinarizer
multi_label = MultiLabelBinarizer(classes=sorted(type_dictionary.keys()))
y_train = multi_label.fit_transform(types)
print(y_train[0])

[0 0 1 1 0]


In [6]:
import time

In [12]:
prompt = """
Decide a Multilabel classification whether an uterance describing the insight type about NBA stats is a set of target labels (Single Player Performance, Whole Game Review, Strategy and Tactic, About Referee, Accident).
Single player performance is about single player.
Whole game review maybe about Overall team performance,Battle Report,team stats.
Strategy and Tactic maybe about formation,Lineups,Offensive tendencies,Tactic tendencies both offence and defence,Some technical and tactical such as:Elevator door,Ghost pick and roll,isolation
About Referee is about malicious fouls,controversial penalties.
Accident is about some extra thing on or out court, such as injury.
For example,
The insight:(The Utah Jazz have helped Lauri Markkanen unlock his offensive potential, becoming the kind of star they build a new foundation around.) is single player performance, Strategy and tactic;
The insight:(Joel Embiid most shots were taken from mid range and happened in first quarter) is single player performance.
Output labels only.
 """

def getLabel(input):
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt = prompt + input,
        temperature=0,
        max_tokens=128,
        frequency_penalty=0,
        presence_penalty=0
    )
    message = response["choices"][0]["text"].strip()
    # print(message)
    time.sleep(1)
    
    return message

虽然并行效率高，但是openAI会限速，不能使用并行代码来处理。

In [13]:
insights = df['Insight'].values

# y_predict = [getLabel(insight).split(", ") for insight in insights]
# y_predict = multi_label.fit_transform(y_predict)

In [14]:
y_predict = []
for insight in insights:
    print(insight)
    while (True):
        try:
            label = getLabel(insight)
            break
        except Exception as exc: 
            print(exc)
            time.sleep(1)
    labels = label.split(", ")
    print(labels)
    y_predict.append(labels)

y_predict = multi_label.fit_transform(y_predict)
print(y_predict[1])

The Utah Jazz have helped Lauri Markkanen unlock his offensive potential, becoming the kind of star they build a new foundation around.
['Single Player Performance', 'Strategy and Tactic']
Division and conference makes it easy for Maverick to win
['the game.\n\nSingle Player Performance', 'Strategy and Tactic']
Kevin Durant had to leave the game after Jimmy Butler landed on his knee on this play.
['Single Player Performance', 'Accident']
The Lakers hustled more, but the Mavericks shooting prevailed and this helps Mavericks won the game.
['Whole Game Review']
Without Kevin Durant, Kyrie Irving is aggressive and shot in the form of an ill-advised 30-footer
['Single Player Performance']
Nikola Jokic made a game-winning three-point jumper with .2 seconds on the clock to secure the victory for Denver Nuggets.
['Single Player Performance']
Brooklyn Nets’ challenge of holding the fort without Kevin Durant became even more challenging.
['Single Player Performance']
After Houston Rockets tore t

d:\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:875: UserWarning: unknown class(es) ['No labels.', 'Strategy and Tactic.', 'than usual.\nSingle Player Performance', 'the game.\n\nSingle Player Performance'] will be ignored
  warnings.warn(


In [15]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
 
def print_evaluation_scores(y_val, predicted):
    accuracy=accuracy_score(y_val, predicted)
    f1_score_macro=f1_score(y_val, predicted, average='macro')
    f1_score_micro=f1_score(y_val, predicted, average='micro')
    f1_score_weighted=f1_score(y_val, predicted, average='weighted')
    print("accuracy:",accuracy)
    print("f1_score_macro:",f1_score_macro)
    print("f1_score_micro:",f1_score_micro)
    print("f1_score_weighted:",f1_score_weighted)

In [16]:
print_evaluation_scores(y_train, y_predict)

accuracy: 0.6626506024096386
f1_score_macro: 0.825167627165843
f1_score_micro: 0.8370044052863436
f1_score_weighted: 0.8336414993495418
